In [ ]:
#This is the process of defining a table. 
#A schema is a group of related tables
#class corresponsing to table with given name, where it is inherting data from 
#Define the table
#Anything above dashes is primary key- what uniquely defines the entry/'entity'
#Anything below is a secondary key, parameters do not depend on parameter_dict, does depend on the name 

#If you call SpikeSortingArtifactDetectionParameters(), 
#I am confused about the functions. Actually called methods I believe? 

#Method called Insert_default. Is this "run"? A method is called to execute on that object. The object being the class/table. 

#So if you just call this table/class,
#The first thing that will happen is that this code will insert an entry into the table with the default/none parameters from #insert default

#Then, theres "get no artifact times"
#Im confused, because it looks like this does the actual filtering somehow. Returns valid interval times, but its just the parameter table. 
#Maybe nothing is actually implemented here, but is defined to be called later on in the SpikeSortingRecording table 
#has something to do with the make method later probably 

@schema #define a schema, which is/ will be a group of related tables 
class SpikeSortingArtifactDetectionParameters(dj.Manual): #table creation, manually adding information to it. This table will hold the parameters for artifact detection. 
    #each set of parameters will have a name and a dictionary that contains the parameters, which will be used in the below function.  
    definition = """
    # Table for holding parameters related to artifact detection
    # Note that
    artifact_parameter_name: varchar(200) #name for this set of parameters 
    ---
    parameter_dict: BLOB    # dictionary of parameters for get_no_artifact_times() function
    """

    def insert_default(self): #This will run if there are no arguments to original class?
        #first, insert the default, none, into the table. this contains the parameters that will likely not have any imapct on data. 
        """Insert the default artifact parameters ('none') with a appropriate parameter dict .
        """
        param_dict = {} #empty dictionary called param_dict
        param_dict['skip'] = True
        param_dict['zscore_thresh'] = -1.0
        param_dict['amplitude_thresh'] = -1.0
        param_dict['proportion_above_thresh'] = -1.0
        param_dict['zero_window_len'] = 30 # 1 ms at 30 KHz, but this is of course skipped
        self.insert1({'artifact_parameter_name': 'none', 'parameter_dict' : param_dict},
                        skip_duplicates=True)
#self is a variable that points to the instance of the method you are working with. 

#Looks like this is a method that will actually find times with artifact and remove them. 
#Artifacts are defined as periods where the absolute amplitude of the signal exceeds one or both specified thresholds on the proportion of channels specified, with the period extended by the zero_window/2 samples on each side


#SO this section will take the parameters (why are they hard coded here?) 
#And will return valid times. 
#DOnt have a primary key? 


#want this function to be tied to tables info
#could go around other dj.manual tables, see what kinds of functions you see in them (do they fetch at the end, etc) 
#HOw functions are set up in python (regarding the hard coded things)- if you dont insert, it will default to the hard coded things  
    #find a way to share notebook for us both to check unestanding 
    
    def get_no_artifact_times(self, recording, zscore_thresh=-1.0, amplitude_thresh=-1.0, 
                              proportion_above_thresh=1.0, zero_window_len=1.0, skip: bool=True):
        #Defines the parameters for artifact detection and their types. 
        """returns an interval list of valid times, excluding detected artifacts found in data within recording extractor.
        Artifacts are defined as periods where the absolute amplitude of the signal exceeds one
        or both specified thresholds on the proportion of channels specified, with the period extended
        by the zero_window/2 samples on each side
        Threshold values <0 are ignored.

        :param recording: recording extractor
        :type recording: SpikeInterface recording extractor object
        :param zscore_thresh: Stdev threshold for exclusion, defaults to -1.0
        :type zscore_thresh: float, optional
        :param amplitude_thresh: Amplitude threshold for exclusion, defaults to -1.0
        :type amplitude_thresh: float, optional
        :param proportion_above_thresh:
        :type float, optional
        :param zero_window_len: the width of the window in milliseconds to zero out (window/2 on each side of threshold crossing)
        :type int, optional
        :return: [array of valid times]
        :type: [numpy array]
        """

        # if no thresholds were specified, we return an array with the timestamps of the first and last samples
        if zscore_thresh <= 0 and amplitude_thresh <= 0:
            return np.asarray([[recording._timestamps[0], recording._timestamps[recording.get_num_frames()-1]]])
       
    #This is where the artifact detection and comparison to thresholds occurs. 
    
    #use the specified window length (how long to zero out on either sde of the artifact based on sampling rate
        
        half_window_points = np.round(
            recording.get_sampling_frequency() * 1000 * zero_window_len / 2)
      
    #User defines proportion of electrodes that have to be above threshold for it to be labeled artifact (but two diff versions of this?)
    nelect_above = np.round(proportion_above_thresh * data.shape[0])
    
        # get the data traces
        data = recording.get_traces()

        # compute the number of electrodes that have to be above threshold based on the number of rows of data
        nelect_above = np.round(
            proportion_above_thresh * len(recording.get_channel_ids()))

        # apply the amplitude threshold (find when the data is above amplitude threshold)
        above_a = np.abs(data) > amplitude_thresh

        # zscore the data and get the absolute value for thresholding
        dataz = np.abs(stats.zscore(data, axis=1))
        above_z = dataz > zscore_thresh #find when datas z scare is above specified threshold
       
    #not sure what ravel does. im guessing that it looks acrross electrodes(?) and finds when the sum of them being over thresh is bigger than neglect_above
        above_both = np.ravel(np.argwhere(
            np.sum(np.logical_and(above_z, above_a), axis=0) >= nelect_above))
        valid_timestamps = recording._timestamps #not sure what this is doing yet.. why are these valid timesteps?
        
        # for each above threshold point, set the timestamps on either side of it to -1
        #So, have a list of valid timesteps, and set times that are "above both" plus or minus window points to negative 1...
        for a in above_both:
            valid_timestamps[a - half_window_points:a +
                             half_window_points] = -1

        #anything that is not -1 is now a valid timestamp,     
        # use get_valid_intervals to find all of the resulting valid times.
        #What is get_valid_intervals? Look into these other arguments 
        #Go find get valid intervals 
        return get_valid_intervals(valid_timestamps[valid_timestamps != -1], recording.get_sampling_frequency(), 1.5, 0.001)


@schema
class SpikeSortingRecordingSelection(dj.Manual):
    definition = """
    # Table for holding parameters for each recording that will be spikesorted
    -> SortGroup
    -> SortInterval
    -> SpikeSortingFilterParameters
    ---
    -> SpikeSortingArtifactDetectionParameters
    -> IntervalList
    -> LabTeam
    """
#SO, this one is computed.
@schema
class SpikeSortingRecording(dj.Computed):
    definition = """
    # Schema for storing recording extractors
    -> SpikeSortingRecordingSelection
    ---
    -> IntervalList.proj(sort_interval_list_name='interval_list_name')
    recording_extractor_path: varchar(1000)
    recording_extractor_object: BLOB # the dictionary that allows kachery to retrieve the extractor
    """
    def make(self, key):
        #First, get recording during valid times
        sort_interval_valid_times = self.get_sort_interval_valid_times(key) # get valid time intervals
        with Timer(label='getting filtered recording extractor', verbose=True): 
            recording = self.get_filtered_recording_extractor(key) # not sure whats going on here
            recording_timestamps = recording._timestamps

        # get the artifact detection parameters and apply artifact detection to zero out artifacts (implementation?) 
        artifact_key = (SpikeSortingRecordingSelection & key).fetch1( #key will be the name.. wilbur, default)
            'artifact_parameter_name')
        artifact_param_dict = (SpikeSortingArtifactDetectionParameters & { #get dictionary of parameters)
                               'artifact_parameter_name': artifact_key}).fetch1('parameter_dict')
        #check to see if we want to do this at all
        if not artifact_param_dict['skip']:
            no_artifact_valid_times = SpikeSortingArtifactDetectionParameters().get_no_artifact_times(
                recording, **artifact_param_dict) #** =unpacking
            # update the sort interval valid times to exclude the artifacts
            sort_interval_valid_times = interval_list_intersect(
                sort_interval_valid_times, no_artifact_valid_times)
            # exclude the invalid times
            mask = np.full(recording.get_num_frames(), True, dtype='bool')
            excluded_ind = interval_list_excludes_ind(
                sort_interval_valid_times, recording_timestamps)
            if len(excluded_ind) > 0:
                mask[interval_list_excludes_ind(
                    sort_interval_valid_times, recording_timestamps)] = False
            recording = st.preprocessing.mask(recording, mask)

        #add the sort_interval_valid_times as an interval list
        tmp_key = {}
        tmp_key['nwb_file_name'] = key['nwb_file_name']
        tmp_key['interval_list_name'] = key['nwb_file_name'] + '_' + \
            key['sort_interval_name'] + '_' + str(key['sort_group_id'])+ \
            key['filter_parameter_set_name'] + '_recording'
        tmp_key['valid_times'] = sort_interval_valid_times;
        IntervalList.insert1(tmp_key, replace=True)

        # store the list of valid times for the sort
        key['sort_interval_list_name'] = tmp_key['interval_list_name']

        # Path to files that will hold the recording extractors


In [1]:
# Questions: Mostly about implementation, the concepts make sense to me. 
# How to “call” methods in the class with specific arguments? 
# The arrows? Why some primary keys (?) have arrows before? 
# Whats def make again? 
# What is a recording extractor? 
# What is actually returned from the original parameters class? It looks like its valid times but its also just parameters… 


In [ ]:
#how to call these things, see whats going on.... a few ways. 
#Take these out of schema setups, but keep orgaization. Keep the functions, etc. dismantle from datajointiness, make familiar. Slack when i have questions 
#getting a recording extractor by using spike sorting code (could use beans data- use a tiny amount of data!! super fast!) 5 seconds! beans will be smaller
#how does current spikesorting work? knowing the order of tables that gets filled in (not detail I went rhough today) - relational diagrams. 
#Is there an updated spikesorting notebook? Might be on slack. Basic high level flow, and understanding where recording extractor is(what is it, data type). How would I use that in these functions?  
#when i get to that point, send a slack mesage 